In [9]:
import pandas as pd

subset = pd.read_csv("/home/pedige/.local/bin/DQ/DQ/Topic0_train_data.csv")
subset = subset[:500]
subset['date'] = pd.to_datetime(subset['date']).dt.date
subset

,Unnamed: 0,id,date,news,label,pct_change,text_length,textlemma,textlengthforlemma,word_count,topic,topic_name,Entities,IsEntity
0,104,nifty_2,2010-01-11,Research and Markets: Epilepsy - Drug Pipeline...,Fall,-0.0093,91,research and market epilepsy drug pipeline ana...,78,11,0,pharmacy,"['research', 'and', 'market', 'e', '##pile', '...",1
1,127,nifty_2,2010-01-11,Weird Science? Big Pharma's Top Three M&A; D...,Fall,-0.0093,108,weird science big pharma top three deal nears ...,80,13,0,pharmacy,"['weird', 'science', 'big', 'p', '##har', '##m...",1
2,186,nifty_2,2010-01-11,Research and Markets: Booming Hospital Service...,Fall,-0.0093,63,research and market booming hospital service m...,57,8,0,pharmacy,"['research', 'and', 'market', 'boom', '##ing',...",1
3,222,nifty_2,2010-01-11,BioCentury Publishes Investor Picks for New B...,Fall,-0.0093,72,biocentury publishes investor pick for new bio...,61,8,0,pharmacy,"['bio', '##cent', '##ury', 'publishes', 'inves...",1
4,396,nifty_2,2010-01-11,Research and Markets: Vertex Pharmaceuticals -...,Fall,-0.0093,70,research and market vertex pharmaceutical swot...,65,8,0,pharmacy,"['research', 'and', 'market', 'vertex', 'pharm...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,42369,nifty_150,2010-10-06,Robbins & Myers to Buy T-3,Neutral,-0.0012,26,robbins myers buy,17,3,0,pharmacy,"['r', '##ob', '##bin', '##s', 'my', '##ers', '...",1
496,42715,nifty_152,2010-10-11,Genesis Biopharma to Present at the Informed I...,Neutral,0.0031,122,genesis biopharma present the informed investo...,106,13,0,pharmacy,"['genes', '##is', 'bio', '##pha', '##rma', 'pr...",1
497,42866,nifty_152,2010-10-11,Allscripts Acute Care Users Conference Explore...,Neutral,0.0031,102,allscripts acute care user conference explores...,96,13,0,pharmacy,"['all', '##s', '##cripts', 'acute', 'care', 'u...",1
498,42911,nifty_152,2010-10-11,Research and Markets: Healthcare IT Market Out...,Neutral,0.0031,138,research and market healthcare market outlook ...,118,15,0,pharmacy,"['research', 'and', 'market', 'healthcare', 'm...",1


In [10]:
import pandas as pd
import torch
from itertools import combinations
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2') #We used this smallest variant of sbert as the starting point)

filtered_entities = subset[subset['IsEntity'] == 1][['Entities', 'date', 'textlemma']] #Here the filtration for 1s from the NER)

def calculate_sbert_similarity(entities1, entities2):
    if not entities1 or not entities2:
        return 0  # Return 0 similarity if either entity list is empty

    text1 = " ".join(entities1)
    text2 = " ".join(entities2)

    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)

    similarity=util.pytorch_cos_sim(embedding1, embedding2).item()
    return similarity

# Compare entity similarities between articles
similarities = []
pairs = list(combinations(filtered_entities.index, 2))

for i, j in pairs:
    entities1 = filtered_entities.loc[i, "Entities"]
    entities2 = filtered_entities.loc[j, "Entities"]

    if len(entities1) > 1 and len(entities2) > 1:
        sim = calculate_sbert_similarity(entities1, entities2)
        date_difference = abs((filtered_entities.loc[i, "date"] - filtered_entities.loc[j, "date"]).days)

        similarities.append({
            "Entity1": entities1,
            "Entity2": entities2,
            "Article_1_News": filtered_entities.loc[i, "textlemma"],
            "Article_2_News": filtered_entities.loc[j, "textlemma"],
            "Date_Difference": date_difference,
            "SBERT_Similarity": sim})

similarity_df = pd.DataFrame(similarities)
high_similarity = similarity_df.query('SBERT_Similarity > 0.8')
print(high_similarity.head())

                                             Entity1  \
0  ['research', 'and', 'market', 'e', '##pile', '...   
1  ['research', 'and', 'market', 'e', '##pile', '...   
2  ['research', 'and', 'market', 'e', '##pile', '...   
3  ['research', 'and', 'market', 'e', '##pile', '...   
4  ['research', 'and', 'market', 'e', '##pile', '...   

                                             Entity2  \
0  ['weird', 'science', 'big', 'p', '##har', '##m...   
1  ['research', 'and', 'market', 'boom', '##ing',...   
2  ['bio', '##cent', '##ury', 'publishes', 'inves...   
3  ['research', 'and', 'market', 'vertex', 'pharm...   
4  ['elite', 'p', '##har', '##ma', 'inside', '##r...   

                                      Article_1_News  \
0  research and market epilepsy drug pipeline ana...   
1  research and market epilepsy drug pipeline ana...   
2  research and market epilepsy drug pipeline ana...   
3  research and market epilepsy drug pipeline ana...   
4  research and market epilepsy drug pipeline 